# Scenario Control


## Introduction

This should serve as a guide on how to load and control scenarios, those ones that are included with the simulation, but also the custom scenarios made with BeamNGpy or BeamNG tools. Let's start by opening the BeamNG simulator.

In [1]:
from beamngpy import BeamNGpy, Scenario

beamng = BeamNGpy("localhost", 25252)
beamng.open()

Now, we will talk a little bit about scenario creation first for you to see which options you have if you want to create your own BeamNG scenario.

## Scenario creation

There are multiple ways how to create scenarios. The recommended ones are to use BeamNGpy, the [Flowgraph Editor](https://documentation.beamng.com/modding/flowgraph/introduction-to-flowgraph/examples/#general) included with BeamNG or the [Mission Editor](https://documentation.beamng.com/modding/gamemodes/missions/), also included with BeamNG. There are also scenarios written using the [legacy system](https://documentation.beamng.com/modding/gamemodes/scenario/), but this way of creation is deprecated and is not recommended for new scenarios.

### BeamNGpy
BeamNGpy supports creation of scenarios which include custom vehicles, roads, checkpoint definitions and more. All the other examples show you how to create BeamNGpy scenarios, see the [examples and the Example Guide](https://github.com/BeamNG/BeamNGpy/tree/master/examples#example-guide) included with BeamNGpy for more information.

Scenarios created in BeamNGpy consists of:
 - `<scenario_name>.json`: scenario metadata
 - `<scenario_name>.prefab.json`: scenario objects

These two files are written by BeamNGpy to the userfolder when calling the `Scenario.make` function.

### Flowgraph Editor

These are scenarios created inside the Flowgraph Editor, which is a powerful node-based scripting language that allows to create dynamic scenarios for BeamNG.drive without coding.

To see an example of a Flowgraph scenario, you can open the `/levels/italy/scenarios/noFuel_v2/gravityFuel2.flow.json` file inside the Flowgraph editor.

After completing your Flowgraph scenario, you can use the `File -> Save as Scenario...` option in the Flowgraph Editor to save a `.json` file including information about the scenario, which can then be loaded by BeamNGpy. There is also a `.flow.json` that included a definition of the graph, but only the metadata `.json` file can be used to load the scenario.

To learn about Flowgraph, please consult the [Flowgraph documentation](https://documentation.beamng.com/modding/flowgraph/introduction-to-flowgraph/examples/#general).

### Mission Editor
With BeamNGpy, you are also able to load the missions created with the mission system. Use the [Missions documentation](https://documentation.beamng.com/modding/gamemodes/missions/) to learn about how to create missions inside BeamNG. There exist premade mission types, but you can also use custom Flowgraph code to provide more complex mission behavior.

Missions are located in the `/gameplay/missions` folder. To load them with BeamNGpy, use the path to the `info.json` file, for example `/gameplay/missions/west_coast_usa/aiRace/002-highway/info.json`.

## Scenario loading

To load a scenario, you need to find the path to its definition. The `beamng.scenario.get_scenarios()` function will help you with that, it returns a list of all scenarios that can be loaded in BeamNG, that includes the built-in scenarios and missions, as well as all the custom scenarios made with either BeamNG or using the tools inside the simulator.

In [2]:
?beamng.scenario.get_scenarios

Signature:
beamng.scenario.get_scenarios(
    levels: 'Iterable[str | Level] | None' = None,
) -> 'Dict[str, List[Scenario]]'
Docstring:
Queries the available scenarios and returns them as a mapping of
paths to :class:`.Scenario` instances. The scenarios are constructed
to point to their parent levels, so to avoid extra queries to the
simulator about existing levels, a cache of available levels can be
passed to this method. If a partial list of levels is supplied, then
only scenarios for these levels will be queried and returned.

Args:
    levels: A list of level names or :class:`.Level` instances to get scenarios
            for. If None, scenarios from all levels will be returned.

Returns:
    A mapping of level names to lists of :class:`.Scenario` instances.
File:      c:\dev\beamngpy\src\beamngpy\api\beamng\scenario.py
Type:      method

In [3]:
scenarios = beamng.scenario.get_scenarios()
print(scenarios.keys())

dict_keys(['jungle_rock_island', 'west_coast_usa', 'johnson_valley', 'GridMap', 'gridmap_v2', 'east_coast_usa', 'italy', 'automation_test_track', 'Utah', 'small_island', 'driver_training', 'hirochi_raceway', 'derby', 'smallgrid'])


Let's check some of the scenarios that are happening on the 'West Coast USA' map.

In [4]:
west_coast_scenarios = scenarios["west_coast_usa"]
print(
    f"There are {len(west_coast_scenarios)} West Coast USA scenarios that can be loaded."
)
west_coast_scenarios[:10]

There are 23 West Coast USA scenarios that can be loaded.


[<Scenario(level='west_coast_usa', name='Shuffleboard Shakedown', path='/gameplay/missions/west_coast_usa/drift/005-Shuffleboard/info.json')>,
 <Scenario(level='west_coast_usa', name='Journey to the Set Location', path='/levels/west_coast_usa/scenarios/busdriver_stunt/busdriver_stunt_follow.json')>,
 <Scenario(level='west_coast_usa', name='Finale: Ballistic Bus', path='/levels/west_coast_usa/scenarios/busdriver_stunt/busdriver_stunt_ballistic.json')>,
 <Scenario(level='west_coast_usa', name='Quarry: Triple Threat', path='/gameplay/missions/west_coast_usa/timeTrial/021-Quarry4/info.json')>,
 <Scenario(level='west_coast_usa', name='Action Chase', path='/levels/west_coast_usa/scenarios/busdriver_stunt/busdriver_stunt_crush.json')>,
 <Scenario(level='west_coast_usa', name='Stolen Steel', path='/gameplay/missions/west_coast_usa/collection/004-Stolensteel/info.json')>,
 <Scenario(level='west_coast_usa', name='Gas Runner', path='/gameplay/missions/west_coast_usa/arrive/005-Gas/info.json')>,
 

The `get_scenarios()` function returns instances of the `Scenario`, so you can use the returned values as an argument to the `beamng.scenario.load` function. You can also create a new instance of the `Scenario` class with the same parameters, if that is more convenient for you.

In [5]:
?beamng.scenario.load

Signature:
beamng.scenario.load(
    scenario: 'Scenario',
    precompile_shaders: 'bool' = True,
    connect_player_vehicle: 'bool' = True,
    connect_existing_vehicles: 'bool' = True,
) -> 'None'
Docstring:
Loads the given scenario in the simulation and returns once loading
is finished.

Args:
    scenario: The scenario to load.
    precompile_shaders: Whether the shaders should be compiled before the start of the scenario.
                        If False, the first load of a map will take a longer time, but disabling
                        the precompilation can lead to issues with the :class:`Camera` sensor.
                        Defaults to True.
    connect_player_vehicle: Whether the player vehicle should be connected
                            to this (:class:``.Scenario``) instance. Defaults to True.
    connect_existing_vehicles: Whether ALL vehicles spawned already in the scenario should be connected
                               to this (:class:``.Scenario``) instanc

## BeamNGpy / Legacy scenarios

Legacy scenarios are defined by their corresponding `.json` file. You can get the paths to the existing scenarios with the `beamng.scenario.get_scenarios()` function, discussed in the [Scenario Loading](#Scenario-loading) section.

You are also able to run any scenario created before with `Scenario.make`, as it uses the same format (discussed in the [Scenario Creation](#Scenario-creation) section).

In [6]:
?Scenario.make

Signature: Scenario.make(self, bng: 'BeamNGpy') -> 'None'
Docstring:
Generates necessary files to describe the scenario in the simulation
and outputs them to the simulator.

Args:
    bng: The BeamNGpy instance to generate the scenario for.

Raises:
    BNGError: If the scenario already has set its info .json file included.
File:      c:\dev\beamngpy\src\beamngpy\scenario\scenario.py
Type:      function

In [7]:
scenario = Scenario(
    level="west_coast_usa",
    name="Journey to the Set Location",
    path="/levels/west_coast_usa/scenarios/busdriver_stunt/busdriver_stunt_follow.json",
)
beamng.scenario.load(scenario)

The `beamng.scenario.load` function automatically loads the vehicles, which are currently part of the scenario, and stores them to the `scenario.vehicles` dictionary, where keys are the IDs of the vehicles.

In [8]:
vehicles = scenario.vehicles
vehicles

{'scenario_player0': <beamngpy.vehicle.vehicle.Vehicle at 0x22a21a75e80>,
 'scenario_crew': <beamngpy.vehicle.vehicle.Vehicle at 0x22a21a75d90>}

As the `connect_existing_vehicles` parameter is set to `True` by default, the vehicles are connected to BeamNGpy after loading the scenario. To not connect them automatically, you can set the `connect_player_vehicle` and `connect_existing_vehicles` arguments in `beamng.scenario.load` to `False`.

In [9]:
player, opponent = vehicles["scenario_player0"], vehicles["scenario_crew"]
assert player.is_connected() and opponent.is_connected(), "Vehicles not connected!"

And the scenario is ready to start!

In [10]:
beamng.scenario.start()
player.control(throttle=1.0)

### Flowgraph scenarios

Flowgraph scenarios are defined by their `.json` and `.flow.json` files. You can get the paths to the existing scenarios with the `beamng.scenario.get_scenarios()` function, discussed in the [Scenario Loading](#Scenario-loading) section.

**There are no pre-existing BeamNG Flowgraph scenarios (they got converted into missions in v0.33), but you can create your own using the Flowgraph Editor and the `File -> Save as scenario...` option.**

In [11]:
scenario = Scenario(
    level="<level>",
    name="<SCENARIO NAME>",
    path="/levels/<level>/scenarios/<scenario name>.json",
)
beamng.scenario.load(scenario)

In [12]:
vehicles = scenario.vehicles
vehicles

{'object_1': <beamngpy.vehicle.vehicle.Vehicle at 0x2365bf90d00>,
 'trailer': <beamngpy.vehicle.vehicle.Vehicle at 0x2365d7a91c0>,
 'object_64657': <beamngpy.vehicle.vehicle.Vehicle at 0x2365d7b96a0>}

In this type of scenario, the name of the player vehicle may be changing every load, if it is not assigned in the Flowgraph scenario. You can get the player vehicle ID dynamically, and then find it in the vehicles dictionary.

In [13]:
player_vid = beamng.vehicles.get_player_vehicle_id()["vid"]
player = vehicles[player_vid]

In [14]:
beamng.scenario.start()
player.control(steering=0.2)

### Missions

Missions are defined by their `info.json` files, which are always located in `/gameplay/missions`. You can get the paths to the existing scenarios with the `beamng.scenario.get_scenarios()` function, discussed in the [Scenario Loading](#Scenario-loading) section.

In [15]:
scenario = Scenario(
    level="east_coast_usa",
    name="Taken Treasure",
    path="/gameplay/missions/east_coast_usa/evade/001-Taken/info.json",
)
beamng.scenario.load(scenario)

In [16]:
vehicles = scenario.vehicles
vehicles

{'clone': <beamngpy.vehicle.vehicle.Vehicle at 0x22a21a75400>}

In [17]:
player = vehicles["clone"]

In [18]:
beamng.scenario.start()
player.ai.set_mode("traffic")
player.ai.set_speed(80 / 3.6)

### Where next

And this is the end of the scenario control guide. If you want to see more examples, please follow the [example guide](https://github.com/BeamNG/BeamNGpy/blob/master/examples/README.md).